Install Library

In [1]:
pip install google-api-python-client pandas


   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.2 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/13.2 MB 5.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.6/13.2 MB 4.3 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/13.2 MB 4.3 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/13.2 MB 4.3 MB/s eta 0:00:03
   ---------------- ----------------------- 5.5/13.2 MB 4.4 MB/s eta 0:00:02
   ------------------- -------------------- 6.3/13.2 MB 4.3 MB/s eta 0:00:02
   --------------------- ------------------ 7.1/13.2 MB 4.3 MB/s eta 0:00:02
   ----------------------- ---------------- 7.9/13.2 MB 4.3 MB/s eta 0:00:02
   -------------------------- ------------- 8.9/13.2 MB 4.3 MB/s eta 0:00:02
   ------------------------------ --------- 10.0/13.2 MB 4.3 MB/s eta 0:00:01
   -------------------------------- ------- 10.7/13.2 MB 4.2 MB/s eta 0:00:01
   -

In [4]:
pip install langdetect


     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     -------------------- ----------------- 524.3/981.5 kB 4.2 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993251 sha256=be3be86644dafc2ff431aaee4c2457deba5dc14ac228cd5c0dbd800c1c43e39d
  Stored in directory: c:\users\rizki\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


 Scraping Komentar YouTube via API

In [ ]:
from googleapiclient.discovery import build
from langdetect import detect
import pandas as pd
import time
from urllib.parse import urlparse, parse_qs

def get_all_comment_texts(video_id, api_key, max_comments):
    youtube = build('youtube', 'v3', developerKey=api_key)
    comments = []
    next_page_token = None

    while True:
        if len(comments) >= max_comments:
            break

        try:
            request = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                pageToken=next_page_token,
                maxResults=100,
                textFormat='plainText'
            )
            response = request.execute()
        except Exception as e:
            print(f"[!] Error saat memuat komentar: {e}")
            break

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            text = comment.get('textDisplay', '')

            # Deteksi bahasa
            try:
                if detect(text) != 'en':
                    continue  # Skip jika bukan bahasa Inggris
            except:
                continue  # Skip komentar yang gagal dideteksi

            comments.append(text)

            if len(comments) >= max_comments:
                break

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

        time.sleep(0.3)  # Hindari rate limit

    return pd.DataFrame(comments, columns=['text'])

def extract_video_id(url):
    query = urlparse(url)
    params = parse_qs(query.query)
    return params["v"][0] if "v" in params else None

def save_only_text_comments(video_url, api_key, output_csv, max_comments):
    video_id = extract_video_id(video_url)
    df = get_all_comment_texts(video_id, api_key, max_comments=max_comments)
    df.to_csv(output_csv, index=False)
    print(f"[✓] Disimpan ke '{output_csv}' dengan {len(df)} komentar berbahasa Inggris (hanya teks)")

# ==== Masukkan API Key dan URL Video ====
API_KEY = Isi API ANDA'
official_trailer_url = 'Isi Link YT'

# Jalankan
save_only_text_comments(official_trailer_url, API_KEY, 'official_trailer_comments.csv', 40000)


[✓] Disimpan ke 'official_trailer_comments1.csv.csv' dengan 3346 komentar berbahasa Inggris (hanya teks)
